## Basic import

In [ ]:
import torch
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import dill
from tqdm import tqdm
from torch.optim.lr_scheduler import ReduceLROnPlateau
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c107WarningC1ENS_7variantIJNS0_11UserWarningENS0_18DeprecationWarningEEEERKNS_14SourceLocationENSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEEb'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


## Load Dataset

### Fashion MNIST

In [ ]:
transform_train_FM=transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((32,32)),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.28604063391685486,0.28604063391685486,0.28604063391685486),(0.10887365788221359,0.10887365788221359,0.10887365788221359))
])

transform_test_FM=transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize((0.28604063391685486,0.28604063391685486,0.28604063391685486),(0.10887365788221359,0.10887365788221359,0.10887365788221359))
])

trainset_FM = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=True, transform=transform_train_FM)

testset_FM = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=False, transform=transform_test_FM)

cal_dataset_FM, test_dataset_FM = torch.utils.data.random_split(testset_FM, [5000, 5000])

train_loader_FM = torch.utils.data.DataLoader(trainset_FM, batch_size=128, shuffle=True)
cal_loader_FM = torch.utils.data.DataLoader(cal_dataset_FM, batch_size=128, shuffle=False)
test_loader_FM = torch.utils.data.DataLoader(test_dataset_FM, batch_size=128, shuffle=False)

class_names_FM = ["T-shirt/top","Trouser","Pullover","Dress","Coat","Sandal","Shirt","Sneaker","Bag","Ankle boot"]

### Cifar-10

In [ ]:
transform_train_Cifar = transforms.Compose([
    transforms.RandomCrop(32,padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test_Cifar = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

trainset_Cifar10 = datasets.CIFAR10(root='~/.pytorch/CIFAR10_data/', download=True, train=True, transform=transform_train_Cifar)

testset_Cifar10 = datasets.CIFAR10(root='~/.pytorch/CIFAR10_data/', download=True, train=False, transform=transform_test_Cifar)

cal_dataset_Cifar10, test_dataset_Cifar10 = torch.utils.data.random_split(testset_Cifar10, [5000, 5000])

train_loader_Cifar10 = torch.utils.data.DataLoader(trainset_Cifar10, batch_size=128, shuffle=True)
cal_loader_Cifar10 = torch.utils.data.DataLoader(cal_dataset_Cifar10, batch_size=128, shuffle=False)
test_loader_cifar10 = torch.utils.data.DataLoader(test_dataset_Cifar10, batch_size=128, shuffle=False)

class_names_CIFAR10 = trainset_Cifar10.classes

Files already downloaded and verified
Files already downloaded and verified


### Cifar-100

In [ ]:
trainset_Cifar100 = datasets.CIFAR100(root='~/.pytorch/CIFAR100_data/', download=True, train=True, transform=transform_train_Cifar)

testset_Cifar100 = datasets.CIFAR100(root='~/.pytorch/CIFAR100_data/', download=True, train=False, transform=transform_test_Cifar)

cal_dataset_Cifar100, test_dataset_Cifar100 = torch.utils.data.random_split(testset_Cifar100, [5000, 5000])

train_loader_Cifar100 = torch.utils.data.DataLoader(trainset_Cifar100, batch_size=128, shuffle=True)
cal_loader_Cifar100 = torch.utils.data.DataLoader(cal_dataset_Cifar100, batch_size=128, shuffle=False)
test_loader_cifar100 = torch.utils.data.DataLoader(test_dataset_Cifar100, batch_size=128, shuffle=False)

class_names_CIFAR100 = trainset_Cifar100.classes

Files already downloaded and verified
Files already downloaded and verified


## Define Models

### Simple CNN

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(12544, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [ ]:
model1=CNN()
model1.to(device);

### Resnet 50

In [ ]:
resnet1=models.resnet50(weights=None)
features=resnet1.fc.in_features
resnet1.fc=nn.Linear(features,10)
resnet1.to(device);

In [ ]:
resnet2=models.resnet50(weights=None)
features=resnet2.fc.in_features
resnet2.fc=nn.Linear(features,10)
resnet2.to(device);

In [ ]:
resnet3=models.resnet50(weights=None)
features=resnet3.fc.in_features
resnet3.fc=nn.Linear(features,100)
resnet3.to(device);

## Train function

In [ ]:
def train(model,train_loader,test_loader,epochs=30):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)

    for epoch in range(epochs):
        print(f"Epoch: {epoch+1}/{epochs}")
        print('-' * 10)

        model.train()
        running_loss = 0.0
        running_acc = 0

        for images, labels in tqdm(train_loader, desc="Training", unit="batch"):
            images = images.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()

            log_ps = model(images)
            _, preds = torch.max(log_ps, 1)
            loss = criterion(log_ps, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            running_acc += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = 100 * running_acc / len(train_loader.dataset)
        print(f'Training Loss: {epoch_loss:.4f} Acc: {epoch_acc:.2f}%')

        model.eval()
        val_loss = 0.0
        val_acc = 0

        with torch.no_grad():
            for images, labels in tqdm(test_loader, desc="Validation", unit="batch"):
                images = images.to(device)
                labels = labels.to(device)

                log_ps = model(images)
                _, preds = torch.max(log_ps, 1)
                loss = criterion(log_ps, labels)

                val_loss += loss.item()
                val_acc += torch.sum(preds == labels.data)

        epoch_val_loss = val_loss / len(test_loader.dataset)
        epoch_val_acc = 100 * val_acc / len(test_loader.dataset)
        print(f'Validation Loss: {epoch_val_loss:.4f} Acc: {epoch_val_acc:.2f}%')
        print('-' * 10)
        scheduler.step(epoch_val_loss)

## Training

In [ ]:
train(model1,train_loader_FM,test_loader_FM)
torch.save(model1, 'model/model1.pth', pickle_module=dill)

Epoch: 1/30
----------


Training: 100%|██████████| 469/469 [00:19<00:00, 24.30batch/s]


Training Loss: 0.0070 Acc: 66.65%


Validation: 100%|██████████| 40/40 [00:01<00:00, 35.74batch/s]


Validation Loss: 0.0041 Acc: 80.40%
----------
Epoch: 2/30
----------


...

Validation Loss: 0.0021 Acc: 90.52%
----------
Epoch: 30/30
----------


Training: 100%|██████████| 469/469 [00:19<00:00, 23.97batch/s]


Training Loss: 0.0027 Acc: 87.22%


Validation: 100%|██████████| 40/40 [00:01<00:00, 36.19batch/s]

Validation Loss: 0.0022 Acc: 90.16%
----------


In [ ]:
train(resnet1,train_loader_FM,test_loader_FM)
torch.save(resnet1, 'model/resnet1.pth', pickle_module=dill)

Epoch: 1/30
----------


Training: 100%|██████████| 469/469 [00:28<00:00, 16.63batch/s]


Training Loss: 0.0066 Acc: 69.92%


Validation: 100%|██████████| 40/40 [00:01<00:00, 28.97batch/s]


Validation Loss: 0.0041 Acc: 80.72%
----------
Epoch: 2/30
----------


...

Validation Loss: 0.0020 Acc: 90.76%
----------
Epoch: 30/30
----------


Training: 100%|██████████| 469/469 [00:26<00:00, 17.63batch/s]


Training Loss: 0.0018 Acc: 91.38%


Validation: 100%|██████████| 40/40 [00:01<00:00, 31.07batch/s]


Validation Loss: 0.0022 Acc: 89.86%
----------


In [ ]:
train(resnet2,train_loader_Cifar10,test_loader_cifar10,epochs=50)
torch.save(resnet2, 'model/resnet2.pth', pickle_module=dill)

Epoch: 1/50
----------


Training: 100%|██████████| 391/391 [00:18<00:00, 20.65batch/s]


Training Loss: 0.0151 Acc: 32.81%


Validation: 100%|██████████| 40/40 [00:00<00:00, 42.76batch/s]


Validation Loss: 0.0216 Acc: 36.34%
----------
Epoch: 2/50
----------


...

Validation Loss: 0.0050 Acc: 79.62%
----------
Epoch: 50/50
----------


Training: 100%|██████████| 391/391 [00:18<00:00, 21.49batch/s]


Training Loss: 0.0034 Acc: 84.86%


Validation: 100%|██████████| 40/40 [00:00<00:00, 45.84batch/s]


Validation Loss: 0.0045 Acc: 81.50%
----------


In [ ]:
train(resnet3,train_loader_Cifar100,test_loader_cifar100,epochs=50)
torch.save(resnet3, 'model/resnet3.pth', pickle_module=dill)

Epoch: 1/50
----------


Training: 100%|██████████| 391/391 [00:18<00:00, 21.37batch/s]


Training Loss: 0.0335 Acc: 7.49%


Validation: 100%|██████████| 40/40 [00:00<00:00, 45.15batch/s]


Validation Loss: 0.0504 Acc: 12.50%
----------
Epoch: 2/50
----------


...

Training: 100%|██████████| 391/391 [00:18<00:00, 20.80batch/s]


Training Loss: 0.0039 Acc: 85.58%


Validation: 100%|██████████| 40/40 [00:00<00:00, 46.11batch/s]


Validation Loss: 0.0272 Acc: 48.74%
----------
Epoch: 48/50
----------


Training:  92%|█████████▏| 359/391 [00:16<00:01, 20.43batch/s]